In [1]:
cd "/content/drive/My Drive/audioToLyrics"

/content/drive/My Drive/audioToLyrics


In [2]:
from audiotolyrics.preprocessing.datacontainer import audioContainer, textContainer
from audiotolyrics.textgenmodel.atlmodel import  audioFeatureExtractor

In [6]:
paths_audio_text = []
for genre in ["rap", "country"]:
  paths_audio_text.append(f"./audioToLyrics_{genre}")

In [7]:
songs = audioContainer(paths_audio_text, sr = 4400, limit=5, song_duration=30)

Number of files in folder: 290
Number of audio tracks with lyrics: 11
Number of files in folder: 157
Number of audio tracks with lyrics: 38


In [8]:
songs.paths

['./audioToLyrics_country/Me First And The Gimme Gimmes - Country Roads-M50lKtaE8rs.mp3',
 './audioToLyrics_country/Mel McDaniel - Louisiana Saturday Night-yljf4Lpj7CI.mp3',
 "./audioToLyrics_country/Country Boy's World - Jason Aldean-5AoK4HB2Ey0.mp3",
 './audioToLyrics_country/Brantley Gilbert - Country Must Be Country Wide-8peBUdlUNmM.mp3',
 './audioToLyrics_country/Florida Georgia Line - Simple-3GeaYy6zlXU.mp3']

In [9]:
songs.load_songs()

In [10]:
text = textContainer(paths_audio_text, songs.audio_with_text, songs.audiowaves)

In [11]:
input_audio, input_text, output_text = text.run_datapipeline()

In [12]:
text.cleaned_lyrics

{'Brantley Gilbert - Country Must Be Country Wide-8peBUdlUNmM.mp3': ['the way your fingers fit in mine',
  'its five plus five not rocket science',
  'this day in time thats hard to find its true',
  'the road were on aint traffic jam',
  'its sunday drive on piece of land',
  'its paradise as long as im with you',
  'chorus',
  'its like one two three',
  'just as easy as can be',
  'just the way you look at me',
  'you make me smile',
  'aint no need to complicate it we both know thats overrated',
  'weve been there its safe to say it aint our style',
  'its just that simple simple',
  'simple as can be',
  'its just that simple simple',
  'simple as can be',
  'verse',
  'we used to live on instagram',
  'worry bout who all gives damn',
  'bout where weve been and where we ended up',
  'then met you and you met me',
  'and all the rest is history',
  'an epiphany that all we need is us',
  'chorus',
  'its like one two three',
  'just as easy as can be',
  'just the way you look at 

In [13]:
input_audio.shape, input_text.shape, output_text.shape

((1072, 1, 132000), (1072, 12), (1072, 394))

In [14]:
text.end_index_songs

[228, 329, 561, 866, 1072]

In [15]:
# Setting some hyperparameters
ks_list = [16, 3, 3]
filter_list = [16, 32, 64]
stride_list = [4,4,4]
laten_dim = 256
embedding_dim = 128

In [16]:
model = audioFeatureExtractor(text.vocab_length, filter_list, ks_list, stride_list, embedding_size=embedding_dim,
                              latent_dim=laten_dim, max_len= text.max_len,
                             input_size=1,seq_len = input_audio.shape[-1])

In [25]:
model.traintextgen(input_audio, input_text, output_text, epochs=50,)

Starting model training for 50 epochs


/content/drive/My Drive/audioToLyrics/audiotolyrics/textgenmodel/atlmodel.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text, audio = torch.tensor(text).type(torch.LongTensor), torch.tensor(audio)


Epoch #1, loss=0.07248672038515318
Epoch #11, loss=0.06460352513172171
Epoch #21, loss=0.05637809278941324
Epoch #31, loss=0.05549265560340941
Epoch #41, loss=0.05496507463312007


In [26]:
word = "money"

In [27]:
model.spitbars(text.tokenizer, input_audio[6], in_text=word)

'money back back down our the off just aint on be willie be'

In [28]:
model.spitbars(text.tokenizer, input_audio[699], in_text=word)

'money we long bout aint time look say hard find find country find'

In [29]:
text.vocab

{'acting',
 'acura',
 'again',
 'aint',
 'akon',
 'all',
 'almost',
 'along',
 'alright',
 'always',
 'an',
 'and',
 'andre',
 'apartheid',
 'appeals',
 'archive',
 'are',
 'arms',
 'asking',
 'ass',
 'at',
 'aunt',
 'away',
 'baby',
 'back',
 'balls',
 'be',
 'bed',
 'been',
 'belong',
 'benjamin',
 'bernie',
 'bet',
 'better',
 'big',
 'bitch',
 'bitches',
 'black',
 'blacker',
 'blast',
 'blowing',
 'boat',
 'bout',
 'break',
 'breeze',
 'bridge',
 'brought',
 'build',
 'burnin',
 'bust',
 'busy',
 'but',
 'by',
 'calls',
 'can',
 'cant',
 'caught',
 'cause',
 'chance',
 'change',
 'charlie',
 'chase',
 'check',
 'childish',
 'choose',
 'chorus',
 'chump',
 'city',
 'cleaned',
 'clippers',
 'come',
 'compare',
 'constantly',
 'cops',
 'country',
 'couple',
 'courtside',
 'custody',
 'dark',
 'daughter',
 'day',
 'days',
 'dead',
 'dick',
 'dicks',
 'did',
 'do',
 'done',
 'dont',
 'door',
 'down',
 'drivin',
 'dusty',
 'else',
 'evening',
 'every',
 'everyone',
 'everything',
 'exte